In [12]:
import sys
import os
import dill
sys.path.append('/Users/jshleap/Playground/Insight/HiddenKeywords')
from scripts.gimmewords import *
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
mock_landing_page = "https://blog.paperspace.com/tag/machine-learning/" 
mock_GKP_result = "/Users/jshleap/Playground/Insight/HiddenKeywords/data/Paperspace_KW_Stats_2019-09-23_at_14_37_49.csv"
max_search = 10 
max_results = 5 
model = 'lda'

In [13]:
df_opt = dict(skiprows=[0, 1], encoding=detect_encoding(mock_GKP_result), sep='\t')
gkp = pd.read_csv(mock_GKP_result, **df_opt)
gkp.loc[:10, ['Keyword', 'Avg. monthly searches', 'Competition', 'Searches: Aug 2019', 'CPC']]

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Keyword,Avg. monthly searches,Competition,Searches: Aug 2019,CPC
0,papersapce,NaN,Low,NaN,NaN
1,paperspace,NaN,Low,NaN,NaN
2,paperspace ai,NaN,Low,NaN,NaN
3,paperspace deep learning,NaN,Low,NaN,NaN
4,paperspace machine,NaN,Low,NaN,NaN
5,paperspace virtual machine,NaN,Low,NaN,NaN
6,paperspace student,NaN,Low,NaN,NaN
7,paperspace machine learning,NaN,Medium,NaN,NaN
8,free cloud deep learning,NaN,Medium,NaN,NaN
9,paperspace free,NaN,Low,NaN,NaN


In [3]:
# I preran the crawler for speedsake
path = os.path.abspath(os.path.join(os.getcwd(), os.path.pardir))
with open(os.path.join(path, 'pages.dmp')) as p, open(os.path.join(path, 'landing.dmp')) as l:
    text = [line for line in p]
    land = ' '.join([line for line in l])
land[:10]

'august dee'

In [ ]:
# I pre-ran the class 
with open(os.path.join(path, 'iw.pkcl'), 'rb') as p:
    iw = dill.load(p)

In [ ]:
print('Corpus Keywords', iw.pre_keywords[0][:10])
print('Landing page Keywords', iw.landing_kw[:10])
print('GKP keywords', iw.gkp.Keyword.head(10))


In [ ]:
bag_of_keywords = set(x[0] for y in iw.pre_keywords for x in y)
keywords_not_gkp = bag_of_keywords.difference(x[0] for x in iw.landing_kw)
print(len(keywords_not_gkp))
print(list(keywords_not_gkp)[:10])

In [ ]:
iw = IdentifyWords(text, mock_GKP_result, land, max_df=0.9, min_df=0.01,
                 max_features=100, n_keywords=10, model='word2vec')


In [ ]:
w2v = iw.word2vec()

In [ ]:
w2v.most_similar(positive='methods optimizations ')


In [ ]:
w2v.build_vocab(new, update=True)

In [ ]:
w2v.train(new, total_examples=w2v.corpus_count, epochs=w2v.epochs)

In [ ]:
text

In [ ]:
iw = IdentifyWords(text, mock_GKP_result, land, max_df=0.9, min_df=0.01, max_features=100, n_keywords=10, 
                   model='lda')
dictionary, corpus, lda = iw.lda()

In [ ]:
# select and article at random from train_df
random_article_index = np.random.randint(len(text))
bow = dictionary.doc2bow(text[random_article_index])
# get the topic contributions for the document chosen at random above
doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=bow)])
# print the top 5 contributing topics and their words
for i in doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")
new_bow = dictionary.doc2bow(land)
new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])
# print the top 8 contributing topics and their words
for i in new_doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")
# we need to use nested list comprehension here
# this may take 1-2 minutes...
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
doc_topic_dist.shape

In [ ]:
iw.keywords

In [ ]:
def jensen_shannon(query, matrix):
    """
    https://www.kaggle.com/ktattan/lda-and-document-similarity
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))


k=10
query, matrix = new_doc_distribution, doc_topic_dist
sims = jensen_shannon(query,matrix) # list of jensen shannon distances
k_top sims.argsort()[:k] 


In [9]:
df = pd.read_csv('/Users/jshleap/Playground/Insight/HiddenKeywords/df_checkpoint.csv', index_col=0)
df

,ad_position_average,ad_position_max,ad_position_min,bid,cpc_average,cpc_max,cpc_min,daily_clicks_average,daily_clicks_max,daily_clicks_min,daily_cost_average,daily_cost_max,daily_cost_min,daily_impressions_average,daily_impressions_max,daily_impressions_min,key,language,loc_id,match
architecture training,1.06,1.00,1.11,100,15.86,17.45,14.27,0.02,0.02,0.02,0.31,0.34,0.28,0.44,0.49,0.40,architecture training,en,2124,exact
enable,1.06,1.00,1.11,100,4.47,4.91,4.02,0.03,0.04,0.03,0.15,0.17,0.14,0.89,0.98,0.80,enable,en,2124,exact
operations,1.06,1.00,1.11,100,15.79,17.37,14.21,0.14,0.16,0.13,2.27,2.50,2.05,4.15,4.56,3.73,operations,en,2124,exact
hyphens,NaN,NaN,NaN,100,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,hyphens,en,2124,exact
generating,NaN,NaN,NaN,100,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,generating,en,2124,exact
exciting,NaN,NaN,NaN,100,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,exciting,en,2124,exact
captured,NaN,NaN,NaN,100,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,captured,en,2124,exact
define,1.06,1.00,1.11,100,7.73,8.50,6.96,0.02,0.02,0.02,0.14,0.16,0.13,0.44,0.49,0.40,define,en,2124,exact
cells,1.06,1.00,1.11,100,3.94,4.33,3.54,0.02,0.02,0.02,0.08,0.08,0.07,0.44,0.49,0.40,cells,en,2124,exact
refers,NaN,NaN,NaN,100,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,refers,en,2124,exact


In [ ]:
%matplotlib inline
df.plot(kind='scatter', x='daily_cost_average', y='cpc_average')

In [ ]:
df.plot(kind='scatter', x='ad_position_average', y='cpc_average')

In [ ]:
df.plot(kind='scatter', x='daily_impressions_average', y='cpc_average')

In [ ]:
df.plot(kind='scatter', x='daily_impressions_average', y='daily_clicks_average')

In [ ]:
df['kw_value'] = (df.daily_impressions_average + df.daily_clicks_average) * (1/df.daily_cost_average)
df

In [ ]:
df.plot(kind='scatter', x='daily_impressions_average', y='kw_value')

In [14]:
import os, sys
path = os.path.abspath(os.path.pardir)
sys.path.append(path)

In [22]:
from scripts.knapsack import *
print(df.shape)
df = df.dropna()
print(df.shape)
values = (df.daily_impressions_average + df.daily_clicks_average) * \
          (1/df.daily_cost_average)
ks = Knapsack(items_names=df.index.to_list(), values=values.to_list(),
              weights=df.daily_cost_average.to_list(), capacity=20,
              solve_type=5, name='Branch_n_bound')
ks.get_results()


(778, 20)
(778, 20)


TypeError: Expecting a list of tuples

In [23]:
items_names=df.index.to_list()[:50]
values=values.to_list()[:50]
weights=df.daily_cost_average.to_list()[:50]
print(items_names)
print(values)
print(weights)
print(ks.capacity)
print(ks.values[:50])
print(len(ks.packed_items))
print(sum(ks.packed_weights))
df[df.index.isin(ks.packed_items[:10])]

['architecture training', 'enable', 'operations', 'define', 'cells', 'host', 'market', 'cases', 'custom', 'end', 'benchmark', 'action', 'update', 'format', 'platforms', 'basic', 'aims', 'transfer', 'international', 'project', 'square', 'latest', 'pytorch', 'multiplying', 'current', 'benchmarking', 'installation', 'environment', 'organization', 'parts', 'optimization', 'announcement', 'theory', 'accessing', 'short', 'debugging', 'bit', 'reached', 'benefits', 'stages', 'annealing', 'demonstration', 'downloaded', 'areas', 'reliable', 'worlds', 'rdp', 'setting', 'detector', 'cpus']
[1.4838709677419357, 6.133333333333334, 1.8898678414096917, 3.2857142857142856, 5.75, 1.54406580493537, 1.400705052878966, 1.676190476190476, 2.0055555555555555, 0.36860068259385664, 1.67849985685657, 2.325, 2.530487804878049, 1.9827737739497275, 1.8709677419354842, 3.0000000000000004, 3.833333333333334, 4.3125, 2.615, 1.4348837209302328, 0.8444171183831894, 9.842105263157896, 1.4974332648870634, 1.6111111111111

,ad_position_average,ad_position_max,ad_position_min,bid,cpc_average,cpc_max,cpc_min,daily_clicks_average,daily_clicks_max,daily_clicks_min,daily_cost_average,daily_cost_max,daily_cost_min,daily_impressions_average,daily_impressions_max,daily_impressions_min,key,language,loc_id,match
